In [1]:
import pandas as pd
import numpy as np
import requests
import json
from get_mail_mailru import main as mail_main
from parser_broker_report import main as parser_main

In [2]:
def fix_split(ticker_list, transactions, transactions_executed):

    share_split_dict = {
        'FXGD': ['2022-02-17', 10],
        'SBMX': ['2021-06-09', 100],
        'FXUS': ['2022-01-24', 100],
        'FXRL': ['2021-11-24', 100],
        'FXRU': ['2022-02-17', 10],
        'FXDE': ['2021-12-15', 100]
    }

    replacement_dict = {
        'VTBE': 'RSHE',
        'RU000A102HB1': 'SU26227RMFS7',
        'RU000A1038V6': 'SU26238RMFS4',
        'RU000A101QE0': 'SU26234RMFS3'
    }

    ticker_list_copy = ticker_list.copy()
    for ticker in ticker_list_copy:

        if ticker in replacement_dict:
            new_ticker = replacement_dict[ticker]
            ticker_list.remove(ticker)
            ticker_list.append(new_ticker)
            transactions_executed.loc[transactions['Код'] == ticker, 'Код'] = new_ticker

    for ticker in ticker_list:

        if ticker in share_split_dict:
            transactions_executed.loc[(transactions_executed['Код'] == ticker) &
                                      (transactions_executed['Дата заключения'] <
                                       share_split_dict[ticker][0]), 'Количество'] =\
                (transactions_executed.loc[(transactions_executed['Код'] == ticker) &
                                           (transactions_executed['Дата заключения'] <
                                           share_split_dict[ticker][0]), 'Количество'] *
                 share_split_dict[ticker][1])


In [ ]:
def get_stock_data_dict(tickers: list, transactions_executed):

    round_numb_3_list = ['LKOH', 'MGNT']
    round_numb_4_list = ['MOEX', 'YDEX']
    round_numb_5_list = ['HYDR', 'AFKS', 'IRAO', 'SBMM']
    round_numb_6_list = ['GAZP', 'MTSS', 'NVTK', 'ROSN', 'SBER', 'CHMF', 'SNGS', 'SBGD', 'SBMX', 'AFKS', 'AFLT', 'RTKM']

    share_amount_dict = {}
    share_price_dict = {}
    share_commission_dict = {}
    for ticker in tickers:

        if ticker in round_numb_3_list:
            round_numb = 3
        elif ticker in round_numb_4_list:
            round_numb = 4
        elif ticker in round_numb_5_list:
            round_numb = 5
        elif ticker in round_numb_6_list:
            round_numb = 6
        else:
            round_numb = 2

        share_frame = transactions_executed[transactions_executed['Код'] == ticker]
        share_list = list(zip(share_frame['Вид'], share_frame['Количество'], share_frame['Сумма'],
                              share_frame['Комиссия Брокера'], share_frame['Комиссия Биржи']))

        share_amount = share_price_avg = share_total_cost = share_commission = 0
        for i in range(len(share_list)):

            share_type = share_list[i][0]
            amount_new = share_list[i][1]
            share_total_cost_new = share_list[i][2]
            share_commission_new = share_list[i][3] + share_list[i][4]

            if share_type == 'Покупка':
                share_amount += amount_new
                share_commission += share_commission_new
                share_total_cost += share_total_cost_new
                share_price_avg = (share_total_cost + share_commission) / share_amount

            elif share_type == 'Продажа':
                share_amount -= amount_new

                if share_amount > 0:
                    share_commission += share_commission_new
                    # share_total_cost += share_commission_new
                    share_total_cost = share_amount * (share_price_avg + share_commission_new / share_amount)
                    share_price_avg = (share_total_cost + share_commission) / share_amount
                    # share_price_avg += share_commission_new / share_amount
                    # share_price_avg = share_total_cost / share_amount
                else:
                    share_commission = 0
                    share_total_cost = 0

            else:
                raise Exception('Неверный вид транзакции')

        share_amount_dict[ticker] = share_amount
        share_price_dict[ticker] = round(share_price_avg, round_numb)
        share_commission_dict[ticker] = round(share_commission, 2)

    return share_amount_dict, share_price_dict, share_commission_dict


In [23]:
share_frame = transactions_executed[transactions_executed['Код'] == 'SBMM']
share_list = list(zip(share_frame['Вид'], share_frame['Количество'], share_frame['Сумма'],
            share_frame['Комиссия Брокера'], share_frame['Комиссия Биржи']))

In [26]:
pd.DataFrame(share_list)

,0,1,2,3,4
0,Покупка,766,9976.00,0.0,2.99
1,Продажа,100,1310.00,0.0,0.40
2,Продажа,500,6588.25,0.0,1.98
3,Покупка,200,2650.60,0.0,0.80
4,Продажа,366,4881.53,0.0,1.46
5,Покупка,40,533.52,0.0,0.16
6,Покупка,80,1067.92,0.0,0.32
7,Покупка,120,1601.88,0.0,0.48
8,Покупка,1060,14156.30,0.0,4.24
9,Покупка,400,5363.80,0.0,1.61


In [4]:
def get_last_prices_dict(tickers: list, bond_tickers: list):

    last_prices_dict = {}
    trading_list = []
    trading_mode_list = ['TQBR', 'TQTF', 'TQCB', 'TQOB', 'TQIR']
    for id_trading in trading_mode_list:

        stocks = 'shares'
        if id_trading in ['TQOB', 'TQCB', 'TQIR']:
            stocks = 'bonds'

        url = (f"https://iss.moex.com/iss/engines/stock/markets/{stocks}/boards/{id_trading}/securities.json?iss.meta=off&iss.only=marketdata&marketdata.columns=SECID,LAST")

        try:
            response = requests.get(url)
        except requests.exceptions.ConnectionError as e:
            raise Exception('Проблема с запросом: ' + url)

        data = json.loads(response.text)['marketdata']['data']
        trading_list.append(data)

    for ticker in tickers:
        for value in trading_list:
            flag = False
            for line in value:
                if ticker == line[0]:

                    if ticker in bond_tickers and line[1]:
                        last_prices_dict[ticker] = round(line[1] * 10, 2)
                    else:
                        last_prices_dict[ticker] = line[1]
                    flag = True
                    break

            if flag:
                break

    return last_prices_dict


In [5]:
def get_coupon_data_dict(tickers: list):

    coupon_dict = {}
    for id_trading in ['TQOB', 'TQCB', 'TQIR']:
        url = (f"https://iss.moex.com/iss/engines/stock/markets/bonds/boards/{id_trading}/securities.json?iss.meta=off&iss.only=securities&securities.columns=SECID,ACCRUEDINT")

        try:
            response = requests.get(url)
        except requests.exceptions.ConnectionError as e:
            raise Exception('Проблема с запросом: ' + url)

        data = json.loads(response.text)['securities']['data']
        len_data = len(data)
        for ticker in tickers:
            for i in range(len_data):
                line = data[i]
                if ticker == line[0]:
                    coupon_dict[ticker] = line[1]

    return coupon_dict


In [173]:
# mail_main()
parser_main()

In [195]:
portfolio = pd.read_csv('portfolio.csv')
portfolio['Дата'] = portfolio['Дата'].astype('datetime64[ns]')
transactions = pd.read_csv('transactions.csv').drop_duplicates(['Дата заключения', 'Время заключения', 'Статус', 'Номер сделки'])
transactions['Дата заключения'] = pd.to_datetime(transactions['Дата заключения'] + ' ' + transactions['Время заключения'], dayfirst=True)
transactions['Дата расчетов'] = pd.to_datetime(transactions['Дата расчетов'], dayfirst=True)
transactions = transactions.drop('Время заключения', axis=1).dropna(axis=1).sort_values('Дата заключения').reset_index(drop=True)
transactions_executed = transactions[transactions['Статус'] == 'И']

In [196]:
ticker_list = list(transactions_executed['Код'].unique())


In [197]:
fix_split(ticker_list, transactions, transactions_executed)
coupon_dict = get_coupon_data_dict(ticker_list)
share_amount_dict, share_price_dict, share_commission_dict = get_stock_data_dict(ticker_list, transactions_executed)
last_prices_dict = get_last_prices_dict(ticker_list, list(coupon_dict.keys()))

portfolio_dict = {
    'Котировки': last_prices_dict,
    'НКД': coupon_dict,
    'Количество': share_amount_dict,
    'Средняя цена': share_price_dict,
    'Комиссия': share_commission_dict
}

main_df = pd.DataFrame.from_dict(portfolio_dict)
main_df.index.name = 'Название'
main_df[(main_df['Количество'] == 0) | (main_df['Котировки'] == '')] = np.nan
main_df.dropna(axis=0, subset=['Котировки', 'Количество'], inplace=True)
main_df['НКД'] = main_df['НКД'].fillna(0)
main_df[['Котировки', 'НКД']] = main_df[['Котировки', 'НКД']].astype('float64')
main_df['Количество'] = main_df['Количество'].astype('int64')
main_df['Средняя цена'] = main_df['Средняя цена']
main_df['Текущая цена'] = round((main_df['Котировки'] + main_df['НКД']) * main_df['Количество'], 2)
main_df['P/L, руб.'] = round(main_df['Текущая цена'] - main_df['Средняя цена'] * main_df['Количество'], 2)
main_df['P/L, %'] = ((main_df['Котировки'] + main_df['НКД']) * 100 / main_df['Средняя цена'] - 100).round(2)


In [198]:
main_df.to_csv(path_or_buf='portfolio_main.csv', index_label=False)

In [199]:
df = pd.read_csv('portfolio_main.csv')
df.index.name = 'Название'

In [12]:
df

,Котировки,НКД,Количество,Средняя цена,Комиссия,Текущая цена,"P/L, руб.","P/L, %"
Название,,,,,,,,
SBMX,18.1790,0.00,1490,17.369671,7.54,27086.71,1205.90,4.66
SBER,318.8700,0.00,150,266.495667,33.45,47830.50,7856.15,19.65
MTSS,234.6500,0.00,100,254.623000,21.30,23465.00,-1997.30,-7.84
GAZP,127.9700,0.00,90,167.589667,13.57,11517.30,-3565.77,-23.64
LKOH,6280.0000,0.00,4,6694.875000,17.50,25120.00,-1659.50,-6.20
MGNT,3669.5000,0.00,7,5301.943000,31.10,25686.50,-11427.10,-30.79
CHMF,1045.0000,0.00,16,1404.878750,18.86,16720.00,-5758.06,-25.62
AFKS,15.4320,0.00,1200,17.475590,18.41,18518.40,-2452.31,-11.69
IRAO,3.2340,0.00,5300,3.977600,17.74,17140.20,-3941.08,-18.69


In [200]:
df

,Котировки,НКД,Количество,Средняя цена,Комиссия,Текущая цена,"P/L, руб.","P/L, %"
Название,,,,,,,,
SBMX,18.0970,0.00,1490,17.369671,7.54,26964.53,1083.72,4.19
SBER,319.2800,0.00,150,266.495667,33.45,47892.00,7917.65,19.81
MTSS,234.4500,0.00,100,254.623000,21.30,23445.00,-2017.30,-7.92
GAZP,127.2600,0.00,90,167.589667,13.57,11453.40,-3629.67,-24.06
LKOH,6241.5000,0.00,4,6694.875000,17.50,24966.00,-1813.50,-6.77
MGNT,3672.0000,0.00,7,5301.943000,31.10,25704.00,-11409.60,-30.74
CHMF,1040.2000,0.00,16,1404.878750,18.86,16643.20,-5834.86,-25.96
AFKS,15.3650,0.00,1200,17.475590,18.41,18438.00,-2532.71,-12.08
IRAO,3.2165,0.00,5300,3.977640,17.74,17047.45,-4034.04,-19.14
